지금까지 우리가 배운 내용을 기초로 tensorflow의 mnist example을 CNN으로 구현해보자! 



In [10]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

# 전처리 단계(결측치, 이상치, 정규화, feature engineering)
# mnist 예제에서는 이미 전처리가 끝난 상태이기 때문에 따로 할게 없다

## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다
# tensorflow 버전에 따라서 dropout을 사용하는 속성이 변경됨 
# 최신버전: rate = 0.8 : drop시키는 노드의 비율
# 이전버전: keep_prob = 0.8 : 살리는 노드의 비율


# 2.2 Convolution Layer
# 입력 데이터는 4차원 
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 
# 1층
W1 = tf.Variable(tf.random_normal([3,3,1,32])) # 3*3, 채널1개(색), 32개의 feature map 생김(->activation map)
L1 = tf.nn.conv2d(x_img, W1, strides=[1,1,1,1],  # 가로방향1칸, 세로방향1칸 
                 padding="SAME") # 패딩 SAME이고 strides를 1,1라고 하면 크기 유지 
L1 = tf.nn.relu(L1) # 값이 너무 커지는 것 방지 
# pooling 넣어도 되고 안넣어도 되고
#  pooling의 목적 -> 이미지 특징을 도드라지게 or 이미지 사이지를 줄이게  둘중하나
L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],
                   padding="SAME")  # 이미지 사이즈가 절반으로 줄어들 것! 
## 나온 결과-> 32개의 채널, 이미지크기는 14

W2 = tf.Variable(tf.random_normal([3,3,32,64]))  # 32개의 채널들어오고, 64개 채널 만들 것이다.
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1],   
                 padding="SAME") 
L2 = tf.nn.relu(L2) 
L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],
                   padding="SAME")    # 이미지 크기 절반으로
## 나온 결과 -> 64개의 채널, 이미지크기는 7


# 학습을 시작해보자!  -> 학습하려면 1차원 데이터로 바꿔야 한다(여러개니까 2차원)
# 2.3 FC layer (모든 레이어에 있는 노드(로지스틱)들이 다음 노드들과 연결되어있는 neural network) (dense layer)
L2 = tf.reshape(L2, [-1,7*7*64])
W3 = tf.get_variable("weight3",shape=[7*7*64,256],  # 256은 내가 임의로 만들어 준 것!
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([256]), name="bias3")
_layer1 = tf.nn.relu(tf.matmul(L2,W3)+b3)
layer1 = tf.nn.dropout(_layer1, keep_prob =keep_rate)  # 과적합 피하기 위해 

W4 = tf.get_variable("weight4",shape=[256,512],  
                    initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]), name="bias4")
_layer2 = tf.nn.relu(tf.matmul(layer1,W4)+b4)
layer2 = tf.nn.dropout(_layer2, keep_prob =keep_rate)  # 과적합 피하기 위해 

W5 = tf.get_variable("weight4",shape=[512,10],  
                    initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]), name="bias5")

logit = tf.matmul(layer2,W5)+b5
H = tf.nn.softmax(logit)

# cost 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=Y))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)  # Adam 쓰려면 learning rate 더 작게 잡아주는게 일반적!


# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습 - epoch, batch 등 

# 정확도 측정

# prediction



Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


## 코드를 더 나은 형태로 바꿔보자! 


In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

# 2.2 Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

L1 = tf.layers.conv2d(inputs=x_img, filters=32,
                     kernel_size=[3,3], padding="SAME",
                     strides=1,  # 가로 세로 방향 같아야 하므로 [1,1]을 줄여서 1로 쓸 수 있다. 
                     activation=tf.nn.relu)  #relu함수 이용해서 activation 시킬거다
# -> 이전 코드
# W1 = tf.Variable(tf.random_normal([3,3,1,32]))                     # filter
# L1 = tf.nn.conv2d(x_img, W1, strides=[1,1,1,1], padding="SAME")    # 합성곱                
# L1 = tf.nn.relu(L1)                                                 # Relu

L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
# -> 이전 코드
# L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1], padding="SAME")   # pooling 작업

L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)
# -> 이전 코드
# L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1], padding="SAME")  

L2 = tf.layers.conv2d(inputs=L1, filters=64,
                     kernel_size=[3,3], padding="SAME",
                     strides=1, activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)



# 2.3 FC layer -> dense layer 

L2 = tf.reshape(L2, [-1,7*7*64])
dense1 = tf.layers.dense(inputs=L2, units=256,   #결과 256개 넘기겠다는 거는 logistic 256개 가지고 있다는것
                         activation=tf.nn.relu)
# -> 이전 코드
# W3 = tf.get_variable("weight3",shape=[7*7*64,256],  # 256은 내가 임의로 만들어 준 것!
#                     initializer=tf.contrib.layers.xavier_initializer())
# b3 = tf.Variable(tf.random_normal([256]), name="bias3")
# _layer1 = tf.nn.relu(tf.matmul(L2,W3)+b3)                   # 행렬곱 -> dense에서 해줌
dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  
    # 계산할 때 배제하는거 -> 중간중간 노드 꺼줘야한다->없어도 되긴 하지만 보통 쓰는게 좋다(overfitting 피하기 위해)

dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
dense3 = tf.nn.dropout(dense3, rate=dropout_rate)

H = tf.layers.dense(inputs=dense3, units=10)  # H에서는 activation 쓰지 않음
# -> 이전 코드
# W5 = tf.get_variable("weight4",shape=[512,10],  
#                     initializer=tf.contrib.layers.xavier_initializer())
# b5 = tf.Variable(tf.random_normal([10]), name="bias5")
# logit = tf.matmul(layer2,W5)+b5
# H = tf.nn.softmax(logit)

# cost 
cost = tf.losses.softmax_cross_entropy(Y,H)
# -> 이전 코드
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)  # 여기서는 adam쓰는게 일반적

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습 - epoch, batch 등 


# 정확도 측정

# prediction



Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


## 위 코드 주석 빼고 적어보자.

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

# 2.2 Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                     strides=1,activation=tf.nn.relu)  
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                     strides=1, activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC layer -> dense layer 
L2 = tf.reshape(L2, [-1,7*7*64])
dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  

dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
dense3 = tf.nn.dropout(dense3, rate=dropout_rate)

H = tf.layers.dense(inputs=dense3, units=10)  # H에서는 activation 쓰지 않음

# cost 
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습 - epoch, batch 등 


# 정확도 측정

# prediction

## 결국 우리의 MNIST 예제는 multinomial 예제! 
### 이미지 1개에 대한 예측값의 H의 도출값은 [0.5, 0.3, 0.2, 0.001, 0.99, 0.44, ...] 
-> 각각 0이 될 확률, 1이 될 확률 등 10개   
가장 큰 곳의 숫자가 예측 숫자가 될 것이다.  
학습 결과는 학습할때마다 달라짐 (초기값 랜덤이라)

## 앙상블(ensemble)은 모델을 여러개 만드는 것! (10개의 모델)

우리가 지금까지 한 것은 한 개의 모델 만드는 과정. 
이미지 1개에 대한 각 모델의 예측값   
Model1(H1) : [0.5, 0.3, 0.02, 0.001, 0.099, 0.044, ...]   
Model2(H2) : [0.4, 0.2, 0.03, 0.011, 0.090, 0.064, ...]   
...
=> SUM => [0.9, 0.5, 0.05, ...]  
각 자리별로 다 더해서 sum 구한다.  
이 SUM을 가지고 prediction 한다.   => 이 값은 0일 것이다. 

즉, 한 사람의 전문가에게 의견을 구하는 것이 아니라 여러 사람의 전문가에게 의견을 구해서 가장 그럴듯한 답을 찾아내는 방식이라고 할 수 있다. 

In [ ]:
# 그냥 한번 돌려보자 (앙상블 X)
# 런타임 -> GPU
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

# 2.2 Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                     strides=1,activation=tf.nn.relu)  
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                     strides=1, activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC layer -> dense layer 
L2 = tf.reshape(L2, [-1,7*7*64])
dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  

dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
dense3 = tf.nn.dropout(dense3, rate=dropout_rate)

H = tf.layers.dense(inputs=dense3, units=10)  # H에서는 activation 쓰지 않음

# cost 
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습 - epoch, batch 등 
num_of_epoch = 30
batch_size = 100

for step in range(num_of_epoch):
    num_of_iter = int(mnist.train.num_examples/batch_size)
    cost_val = 0
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)        
        _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
                                                       Y:batch_y,
                                                       dropout_rate:0.3}) # 30% 끄고 학습해라
    if step%3==0:
        print("cost:{}".format(cost_val))


In [ ]:
# accuracy 측정
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

print("정확도:{}".format(sess.run(accuracy, feed_dict={X:mnist.test.images,
                                                       Y:mnist.test.labels,
                                                       dropout_rate:0})))

In [ ]:
# prediction
result = sess.run(H, feed_dict={X:test_df})

predict = tf.cast(result > 0.5,dtype=tf.int32)
result = sess.run(predict)

In [ ]:
# 앙상블 해보자. 에러!!@!@!@
# 런타임 -> GPU
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

# 2.2 Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                strides=1,activation=tf.nn.relu)  
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                strides=1, activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC layer -> dense layer 
L2 = tf.reshape(L2, [-1,7*7*64])
dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  

dense2 = tf.layers.dense(inputs=dense1, units=512, activation=tf.nn.relu)
dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

H = tf.layers.dense(inputs=dense2, units=10)  # H에서는 activation 쓰지 않음

# cost 
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())


# 학습 - epoch, batch 등 

for n in range(2):
    # 그래프 reset
    tf.reset_default_graph() 

    num_of_epoch = 5
    batch_size = 100

    # 모델은 생성됐으니까 H만 돌려보자.
    for step in range(num_of_epoch):
        num_of_iter = int(mnist.train.num_examples/batch_size)
        cost_val = 0
        for i in range(num_of_iter):
            batch_x, batch_y = mnist.train.next_batch(batch_size)        
            _, cost_val,H_val = sess.run([train,cost,H],feed_dict = {X:batch_x,
                                                        Y:batch_y,
                                                        dropout_rate:0.3}) # 30% 끄고 학습해라
    print(cost_val)
    # test 데이터로 돌려주자 
    H_val = sess.run(H, feed_dict={X:mnist.test.images})

    # 나온 H 값들을 각 자리마다 더해준다.
    if n==0:  # 초기 ndarray 지정
        H_result = H_val
    else:
        H_result = np.add(H_result,H_val)  # 각 자리의 합을 구해주는 함수
    
predict_result = np.argmax(H_result,1)
true_result = np.argmax(mnist.test.labels,1)
correct = np.equal(predict_result,true_result)
accuracy = np.mean(correct)
print(accuracy)


In [ ]:
#### 준원 ####


# Model 정의(Tensorflow graph 생성)
tf.reset_default_graph()

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype = tf.float32)


# 2.2 FC Layer(dense layer)
# 입력데이터는 4차원
x_img = tf.reshape(X, [-1,28,28,1])
L1 = tf.layers.conv2d(inputs = x_img,
                      filters = 32,
                      kernel_size = [3,3],
                      padding = "SAME",
                      strides = 1,
                      activation = tf.nn.relu)

L1 = tf.layers.max_pooling2d(inputs = L1, 
                             pool_size = [2,2], 
                             padding = "SAME",
                             strides = 2)

L1 = tf.layers.dropout(inputs = L1,
                       rate = dropout_rate)


L2 = tf.layers.conv2d(inputs = L1,
                      filters = 64,
                      kernel_size = [3,3],
                      padding = "SAME",
                      strides = 1,
                      activation = tf.nn.relu)

L2 = tf.layers.max_pooling2d(inputs = L2, 
                             pool_size = [2,2], 
                             padding = "SAME",
                             strides = 2)

L2 = tf.layers.dropout(inputs = L2,
                       rate = dropout_rate)



# 2.3 FC Layer (dense layer)
L2 = tf.reshape(L2, [-1,7*7*64])

dense1 = tf.layers.dense(inputs = L2,
                         units = 256,
                         activation=tf.nn.relu)

dense1 = tf.layers.dropout(inputs = dense1,
                           rate = dropout_rate)


dense2 = tf.layers.dense(inputs = dense1,
                         units = 128,
                         activation=tf.nn.relu)

dense2 = tf.layers.dropout(inputs = dense2,
                           rate = dropout_rate)


dense3 = tf.layers.dense(inputs = dense2,
                         units = 512,
                         activation=tf.nn.relu)

dense3 = tf.layers.dropout(inputs = dense3,
                           rate = dropout_rate)


H = tf.layers.dense(inputs=dense3, units=10)


# cost
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(10):
    tf.reset_default_graph()
    num_of_epoch = 30
    batch_size = 100
    
    for step in range(num_of_epoch):
        num_of_iter = int(mnist.train.num_examples / batch_size)
        cost_val = 0
        for j in range(num_of_iter):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _,H_val,cost_val = sess.run([train,H,cost], feed_dict={X : batch_x,
                                                       Y : batch_y, 
                                                       dropout_rate : 0.3})
    print(cost_val)    
    H_val = sess.run(H, feed_dict={X: mnist.test.images})
    if i == 0:
        H_result = H_val
    else:
        H_result = np.add(H_result,H_val)

predict = np.argmax(H_result,1)
Y = mnist.test.labels
correct = np.equal(predict, np.argmax(Y,1))
accuracy = np.mean(correct)
print(accuracy)


In [ ]:
#### 지수 ####

# 런타임 -> GPU
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

H_list = []

for p in range(10):
    ## Model 정의 (Tensorflow graph 생성)
    tf.reset_default_graph()  # 그래프 reset

    # 2.1 placeholder
    X = tf.placeholder(shape=[None,784], dtype=tf.float32)
    Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
    dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

    # 2.2 Convolution Layer
    x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

    L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                        strides=1,activation=tf.nn.relu)  
    L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
    L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

    L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                        strides=1, activation=tf.nn.relu) 
    L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
    L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

    # 2.3 FC layer -> dense layer 
    L2 = tf.reshape(L2, [-1,7*7*64])
    dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
    dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  

    dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
    dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

    dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
    dense3 = tf.nn.dropout(dense3, rate=dropout_rate)

    H = tf.layers.dense(inputs=dense3, units=10)  # H에서는 activation 쓰지 않음

    # cost 
    cost = tf.losses.softmax_cross_entropy(Y,H)

    # train
    train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

    # Session & 초기화
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # 학습 - epoch, batch 등 
    num_of_epoch = 30
    batch_size = 100

    for step in range(num_of_epoch):
        num_of_iter = int(mnist.train.num_examples/batch_size)
        cost_val = 0

        for i in range(num_of_iter):
            batch_x, batch_y = mnist.train.next_batch(batch_size)        
            _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
                                                             Y:batch_y,
                                                             dropout_rate:0.3}) # 30% 끄고 학습해라
    result = sess.run(H, feed_dict={X: mnist.test.images,
                                    Y: mnist.test.labels,
                                    dropout_rate: 0})
    print(result)
    H_list.append(result)

In [ ]:
H_array = np.array(H_list)
print(H_array.shape)

k = 0
for w in range(10):
    t = H_array[w]
    k = k + t
print (k.shape)  

predict = tf.argmax(k,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32 ))

sess.run(accuracy, feed_dict = {X: mnist.test.images,
                                Y: mnist.test.labels,
                                dropout_rate: 0})